In [7]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from scipy.stats import randint as sp_randint
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, Imputer

In [8]:
matplotlib.rcParams['figure.figsize'] = [12.0, 8.0]

In [9]:
data_path = '../data/Auto1-DS-TestData.csv'
random_state = 0
test_size = 0.33

# Read and prepare data

In [10]:
# '?' is used in the file for missing values 
data = pd.read_csv(data_path, na_values='?')

In [11]:
# Check number of missing values in the column we are trying to predict (price)
print('Number of missing value in price column: {}'.format(data.price.isna().sum()))

Number of missing value in price column: 4


In [12]:
# Drop rows with missing values in the price column
data = data.dropna(subset=["price"])

Let's split data into model selection and test. We will only use test data at the end, to report results

In [13]:
tr, te = train_test_split(data, test_size = test_size, random_state = random_state)

We will keep only numeric columns for now, to keep things simple

In [14]:
tr = tr.select_dtypes(include=['int64', 'float64'])

Let's check number of nans in each column

In [15]:
tr.apply(lambda c: c.isnull().sum(), axis=0)

symboling             0
normalized-losses    20
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-size           0
bore                  2
stroke                2
compression-ratio     0
horsepower            2
peak-rpm              2
city-mpg              0
highway-mpg           0
price                 0
dtype: int64

Column normalized-losses has a somewhat large number of nans. It's not extremely large though, so we won't discard that column.

# Train and test different models

Our goal is to approximate the function that maps _feature_ vectors (vectors $v$ drawn from the 14 dimensional space where $v_1$ is the symboling value, $v_2$ is the normalized losses value, etc.) to prices (the _target_ variable).

Price is a continuos variable, so this is a _regression_ problem. 

Our learning algorithm will try to fit a certain model to the training data. That is, based on the relation between the features and the target on the training instances, it will try to adjust certain model parameters so the model captures that relation.

## Ridge regressor

Our first model will be a __Ridge__ regressor. This is a very basic model, that simply adds a regularization term to classic linear regression. It is a _linear_ model. This means that if the underlying function we are trying to approximate (the function that maps features to prices) is highly non linear (i.e price does not vary linearly with the features), the approximation might not be good. However, this models are simple to train, and it is usually a good idea to try them first to get an initial baseline. Further, in many real scenarios where many more features might be available, assuming linear relations between the features and the target is more than reasonable.

For the training algorithm to work, training features must be free of nans. We will impute missing values in a given column with the median for that column. 

Also, linear models tend to work much better when the different features are in the same scale (If column A contains values an order of magnitued larger than values of column B, changes in B might seem irrelevant to the linear function $w_A * A + w_B * B$). We therefore satandarize each column (that is, we substract the column mean and divide by the column standard deviation) 

We assemble all three steps (imputing, scaling and the ridge regressor) in a pipeline. This is extremly useful for organizing the train-validation-test regime, as we will se below.

In [16]:
pipeline = Pipeline([('imputer', Imputer(strategy='median')), ('scaler', StandardScaler()), ('rgs', Ridge())])

Every machine learning model has a certain set of hyper-parameters that need to be chosen to univocally define the model. 

Ridge has only one hyper-parameter that is relevant to us for this problem: the weight of the regularization term. Regularization is a topic in its own. Let's just say that regularization controls how well the trained model will fit the trained data. Less regularization will result in models that fit the training data "too well", possibly capturing artifacts do to noise or sampling (training in data is just a sample of underlying distributions that we are trying to model). Such _overfitted_ models won't _generalize_ well (they won't behave as the underlying mapping function on data not present in the training set)

To choose the right value of the reguarization weight, we perform a grid search.

In [17]:
alpha_range = np.array([10, 1, 0.1, 0.01, 0.001, 0.0001, 0])
grid = {'rgs__alpha':alpha_range}
ridge_gs = GridSearchCV(pipeline, grid, scoring='neg_mean_absolute_error', cv = 5)

Calling $fit$ on the grid search will compute a certain performance measure for each combination of values in the arrays in $grid$ (i.e., for each point in the grid). The combination of values that give the best performance will be chosen as the optimal hyper-parameter set. 

How is this performance measure computed for each point in the grid? By n-fold cross-validation. Specifically, the training set is splitted into as many (equally sized) parts as stated by the variable _cv_ above and, for each possible combination of _cv_ - 1 parts, the pipeline is trained on the set made of those _cv_ - 1 parts and tested on the remaining part. Testing involves using the trained pipeline to _predict_ a value for eaach test vector on the test part. After obtaining a prediction for each vector, a score is computed for the test part by comparing the predictions and the true values (in our case, the actual value in the $price$ column). The performance for a given grid point is obtained by averaging the score of the different test parts. Note that the whole pipeline (imputer, scaler and regressor) is trained each time on the selected _cv_ - 1 parts and tested on the remaining part. That is, not only the parameters of the Ridge regressor are adjusted during each training step: the median (for the imputer) and the mean and standard deviation (for the scaler) are also computed, and then applied to preprocess the test part before applying the trained regressor. Sklearn framework makes this plumbing really easy. 

The specific scoring function is indicated by the parameter $scoring$. In this case, I have chosen the _mean absolute error_ or _MAE_. This is just the average over the test part of the absolute differences between the prediction and the true value. We have chosen MAE because it shows errors in the same units as the price (as opposed to, for example, the _coefficient of determination_): we wanted to see the error in money units. Among the alternatives, we could have also used (for example) _mean suqared error_ (_MSE_). _MSE_ tends to give higher weight to large errors (because of the squared differences), and its interpretation is more involved than that of _MAE_. In any case, most of these scoring functions would probably give similar results in terms of model selection.

In [18]:
ridge_gs.fit(tr.drop(['price'], axis=1), tr.price)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('rgs', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'rgs__alpha': array([1.e+01, 1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 0.e+00])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

## Gradient boosting regressor

As mentioned above, being a linear model, the Ridge regressor might be limited in terms of its ability to capture more complex (non-linear) relations between the different features and the price. Therefore, let's try an example of a non-linear model. 

We have chosen the __Gradient Boosting__ regressor. As it name indicates, this is an example of a _boosting_ method. This means that training this model consists in iteratively training weak learners, each newly trained learner focused in instances that previously trained learners found difficult. In a nutshell, a gradient boosting regressor trains regression trees, and each new tree is trained on the previous trees errors, effectively yielding a steepest descent algorithm on the gradient of the squared loss of the final classifier. 

In [19]:
pipeline = Pipeline([('imputer', Imputer(strategy='median')), 
                     ('rgs', GradientBoostingRegressor(n_estimators=3000, random_state=random_state))])

Note that this time our pipeline does not include a scale step. This is because gradient boosting is based on trees, which are not affected by features scales (as opposed to a linear function)

Compared to the ridge regressor, gradient boosting has many more hyper-parameters that need to be tuned. 

Therefore, instead of doing an exhaustive grid search, this time we will do a randomized search. For each parameter, we specify either a list of possible values or distribution to sample from. Further, we specify a number of grid points to try. Calling $fit$ will try that number of points by sampling from the different distributios/lists. In this way, we can get a relatively (it might take a couple of minutes) fast search while still exploring representaive regions of the space of hyperparameters. Also, note that we set n_jobs = -1, to use every processor available.

In [20]:
learning_rate_vals =  [0.1, 0.05, 0.02, 0.01]
max_depth_dist = sp_randint(4, 6)
min_samples_leaf_dist = sp_randint(3, 17)
max_features_vals = [1.0, 0.3, 0.1]
grid = {'rgs__learning_rate': learning_rate_vals, 
        'rgs__max_depth': max_depth_dist, 
        'rgs__min_samples_leaf': min_samples_leaf_dist, 
        'rgs__max_features': max_features_vals}

# run randomized search
n_points = 20
gb_gs = RandomizedSearchCV(pipeline, param_distributions= grid, 
                           n_iter= n_points, 
                           random_state = random_state,
                           scoring='neg_mean_absolute_error',
                           cv = 5,
                           n_jobs = -1)

In [21]:
gb_gs.fit(tr.drop(['price'], axis=1), tr.price)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('rgs', GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impur...ors=3000, presort='auto', random_state=0,
             subsample=1.0, verbose=0, warm_start=False))]),
          fit_params=None, iid=True, n_iter=20, n_jobs=-1,
          param_distributions={'rgs__max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb1cc72ccd0>, 'rgs__max_features': [1.0, 0.3, 0.1], 'rgs__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb1cc72cc90>, 'rgs__learning_rate': [0.1, 0.05, 0.02, 0.01]},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          return_train_score='warn', scoring='neg_mean_absolute_error',
   

## Including categorical features

Let's try including some categorical features now (we did not take them into account for the previous models). Let's see if it helps improving our ridge regressor performance. In order to use categorical features with this kind of model, we need to _one hot encode_ them. That is, each categorical feature $f$ needs to be represented as $n_f$ different binary features, where $n_f$ is the number of possible values that $f$ can take. Each possible value of $f$ is assigned a different feature among the $n_f$. Given a specific feature vector $x$ with $x_f = v$, one hot encoding $x$ gives as a result a new vector for which $x_f$ is replaced by $n_f$ binary features. Among those features, only the one corresponding to $v$ is set to one (the _hot_ one). The rest of the features are set to zero. 

Pandas function _get__dummies()_ comes in very handy for this. 

In [22]:
data_cat = pd.get_dummies(data)

Let's take a look for example at the categorical feature _fuel-system_. Before one hot encoding, our dataset contained a single column that could take 8 different values:

In [23]:
data['fuel-system'].value_counts()

mpfi    92
2bbl    64
idi     20
1bbl    11
spdi     9
4bbl     3
spfi     1
mfi      1
Name: fuel-system, dtype: int64

After encoding, _fuel-system_ has been replaced by 8 binary columns, one per possible value _fuel-system_ could take:

In [24]:
data_cat[[col for col in data_cat if col.startswith('fuel-system')]].head(8)

,fuel-system_1bbl,fuel-system_2bbl,fuel-system_4bbl,fuel-system_idi,fuel-system_mfi,fuel-system_mpfi,fuel-system_spdi,fuel-system_spfi
0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,0,0
2,0,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0,0
5,0,0,0,0,0,1,0,0
6,0,0,0,0,0,1,0,0
7,0,0,0,0,0,1,0,0


Let's split our data again (using the same random_state to get the same partition as before)

In [25]:
tr, te = train_test_split(data_cat, test_size = test_size, random_state = random_state)

Now let's tune the same pipeline as we did above for the ridge regressor, only this time we include categorical features.

In [26]:
pipeline = Pipeline([('imputer', Imputer(strategy='median')), ('scaler', StandardScaler()), ('rgs', Ridge())])
alpha_range = np.array([10, 1, 0.1, 0.01, 0.001, 0.0001, 0])
grid = {'rgs__alpha':alpha_range}
ridge_cat_gs = GridSearchCV(pipeline, grid, scoring='neg_mean_absolute_error', cv = 5)

In [27]:
ridge_cat_gs.fit(tr.drop(['price'], axis=1), tr.price)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('rgs', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'rgs__alpha': array([1.e+01, 1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 0.e+00])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

## Performance comparison

Let's compare the performance of the different experiments. For each of the three experiments, we add the average score and the score standard deviation to a Pandas dataframe.

In [28]:
performances = pd.DataFrame(columns=['Experiment', 'MAE mean', 'MAE std'])
gs_results = [('Ridge num', ridge_gs), ('Gradient boosting', gb_gs), ('Ridge cat', ridge_cat_gs)]
for r in gs_results:
    std_score = r[1].cv_results_['std_test_score'][r[1].cv_results_['mean_test_score'] == r[1].best_score_][0]
    mean_score = -r[1].best_score_
    performances = performances.append({'Experiment':r[0], 'MAE mean':mean_score, 'MAE std':std_score}, 
                                       ignore_index=True)

In [29]:
performances.sort_values(by='MAE mean')

,Experiment,MAE mean,MAE std
1,Gradient boosting,1468.580398,293.804816
2,Ridge cat,1637.133569,197.356747
0,Ridge num,2248.668911,329.195607


We can see that gradient boosting using only numerical features achieves the best mean MAE. However, the ridge regressor on both numerical and categorical features achieves a somewhat similar mean MAE, and better MAE standard deviation. Ridge regressor trained just on the numerical features is the worse, both in terms of MAE mean and MAE std. 

Which of _gradient boosting_ and _ridge cat_ should we choose?. We choose the ridge regressor, given its simplicity, reasonable MAE mean and lower variance, but it is not easy to get a definite answer here. 

Let's get an estimate of the generalization error for _ridge cat_, by computing MAE mean on the test set.

In [30]:
print('Ridge regressor mean MAE on the test set: {}'.format(
    -ridge_cat_gs.score(te.drop(['price'], axis=1), te.price)))

Ridge regressor mean MAE on the test set: 2359.02668398


# Possible improvements

Below is a list of things that would be interesting to try.

1) Traim gradient boosting with both numerical and categorical features

2) Perform bootstrapping or nested cross-validation to get confident intervals for the estimated generalization error.

3) Analyze feature importance for the different models. Are there any key features? Do they provide useful insights into the use case?

4) Extend these features with an external dataset and check for improvements. 